In [1]:
import json
import os
from glob import glob
from tqdm import tqdm
from collections import defaultdict
import pickle
import argparse

In [2]:
indir = os.path.join('data','preprocessed')
outdir = os.path.join('data','preprocessed')
outfile = os.path.join('data','preprocessed','cofea_full_index.dict')
files = sorted(glob(os.path.join(indir, '*_tokenized.jsonlist')))

In [3]:
index = defaultdict(set)
# collect index of tokens in the documents
for i,file in enumerate(files):
    with open(file) as f:
        docs = f.readlines()
    for x,doc in enumerate(tqdm(docs)):
        doc = json.loads(doc)
        tokens = doc['tokens']
        doc_index = 0
        token = []
        token_index = 0
        while doc_index < len(tokens):
            if doc_index == 0:
                token = tokens[doc_index]
            elif tokens[doc_index].startswith('##'):
                token += tokens[doc_index]    
            else:
                #save the current token and its start index
                index[token].add((i,x,token_index))
                #start a new token
                token = tokens[doc_index]
                token_index = doc_index
            #save the final token if we are at the end   
            if doc_index == len(tokens)-1:
                index[token].add((i,x,token_index))
            
            #update the index based on the broken up tokens, not the whole word             
            doc_index += 1 

100%|██████████████████████████████████| 174686/174686 [12:32<00:00, 232.29it/s]


In [4]:
with open(outfile,'wb') as f:
    pickle.dump(index,file=f)